![figure](lab7/politecnico_h-01.png)

# **Programmable Electronics**
#### Bachelor's Degree in Electrical and Computer Engineering

## Lab Work 7 - VGA Controller 

-----------------------


## Introduction ##
In this work we propose the phased implementation of a pong game using VGA controllers based on the mapped and fabric-mapped schemes. This document does not provide usage details for Vivado. Whenever you have doubts at this level, consult the Lab1 statement.


### Objectives ###

In this work we will test sequentially:
* Varrimento do VGA monitor.
* Object-mapped schema.
* Animated object-mapped scheme.
* Tile-mapped scheme for text.
* Full PONG game

In the instructions below **{sources}** refers to `C:\Xilinx\EEC_EP\sources` and **{labs}** refers to `(C:\Xilinx\EEC_EP\labs)`


_______________________

## Step 1 - Static VGA Interface  ##

### Step 1.1 - Scanning the VGA Monitor ### 

To scan a VGA monitor, it is necessary to implement a synchronization module that generates the horizontal and vertical scanning signals. The VHDL code for this circuit is provided in the **vga_sync.vhd** file. To verify that the circuit works as expected, the RGB input of this circuit is connected to the Basys3 switches. The switches must allow defining the screen color, among the 4096 possible combinations (there are 12 bits of color). The VHDL code of this test circuit is provided in **vga_sync_test.vhd**.


<div>
<img src="lab7/fig1.png" width=700/>
</div>
                                

* Start by downloading the provided source files (Github or Moodle) and saving them in `{sources}/lab7`.


* Open Vivado from the icon on your desktop or: `Start > All Programs > Xilinx Design Tools > Vivado 2022.1`. 


* Select *Create Project* to start the wizard. In the *Create A New Vivado Project* dialog box, select **Next**.


* Click the *Browse* button in the project location field of the new project form, navigate to `c:/Xilinx/EEC_EP/labs` and click **Select**. Enter lab7 in the project name field. Make sure the *Create Project Subdirectory* box is checked. Click **Next**.


* Select the **RTL Project** option in the Project Type form and click **Next**. Make sure the *Do not specify fonts at this time* box is checked.


* In the *Default Part* form, select the **Boards** option and choose the Basys3 board. Click **Next** and then **Finish** to create the Vivado project.


* In the *Sources* panel, select *Add Sources* and follow the necessary steps (*Add or create design sources*) to add all the necessary VHDL files (described above).


* Open the files in Vivado, look at the code and check that they correspond to the modules represented in the figure above. 


* The top file (**vga_sync_test.vhd**) instantiates a module (**clk_wiz_0**) that does not exist in the project. This is a clock management module, which generates a 50MHz signal from the 100MHz input. To add it to the project, choose the IP Catalog option in the Flow Navigator window, as shown in the figure, and double-click on the *Clocking Wizard*.


<div>
<img src="lab7/fig2.png" width=700/>
</div>


* In the *Clocking Wizard* configuration window, leave the default name (clk_wiz_0) and:

    * In the Clocking Options tab, choose the PLL primitive.
    * In the Output Clocks tab, configure a Requested Output Freq (MHz) as 50.
    * Click **OK**.


* Note now that the module appears correctly instantiated in the **vga_test** module hierarchy .

* Now add the constraints file, define the input/output pins and implement the circuit in Basys3.

* Test the circuit by connecting a VGA monitor to the Basys3 board. Notice the different colors on the screen created by the various switch input combinations.



### Step 1.2 - Static object-mapped schema ### 

Let's give an example of generating objects on a VGA monitor, starting by creating an environment with fixed rectangular objects. Each object is described by its coordinates. For example, the wall of the figure can be defined as 4 pixels wide between columns 32 and 35. The coordinates are then WALL_X_L=32 and WALL_X_R=35. It is also necessary to have a signal that defines the color of the wall (wall_rgb) and another that indicates that the object should be sent to the screen (wall_on). This is asserted whenever the horizontal scan is in the wall region.

 

<div>
<img src="lab7/fig3.png" width=800/>
</div>
            
            
For the padle we have something similar, but now we also have to include borders on the Y axis. The bar must have a length of 72 pixels (defined in BAR_Y_SIZE) and must be in the middle of the screen, so the coordinate of the top of the bar is half the maximum value of Y minus half the length of the bar (480/2 - 72/2). The base coordinate is equal to the top coordinate plus the length of the bar. The ball code can be done in the same way.


<div>
<img src="lab7/fig4.png" width=500/>
</div>
                                   

Finally, a multiplexer that examines the ON signals of each object and sends the corresponding RGB signal to the monitor . Combining this circuit with the previous one, we can then visualize the objects on the screen.

* Add to the project the files that allow you to generate these objects statically on the screen (***pong_graph_st.vhd** and **pong_top_st.vhd**).

* Open the files and analyze the code. Observe how the coordinates of the objects are generated.

* Set **pong_top_st.vhd** as the top module (*Set as Top*), and change the input/output pins file (based on the input and output signals of this entity).

* Implement the circuit, program Basys3 and observe the result on the screen.


______

## Step 2 - Dynamic VGA Interface ##

### Step 2.1 - Non-rectangular and animated objects ### 

To implement an object that is not rectangular, the best way is to specify the pattern in a bitmap and generate the **RGB** and **on** signals according to the map. For example, if we want to draw a round ball, we define a square of pixels where the round object is defined. This matrix would then be stored in an 8 by 8 pixel ROM.

To facilitate the animation of objects defined in this way, the signals that define the object's boundaries should not be constants but rather input signals, as shown in the figure. It is then necessary to check whether the scan is in the ball map zone and map the current pixel to the memory address.


<div>
<img src="lab7/fig5.png" width="800"/>
</div>

**Note**: It is also possible to use three separate ROMs to store RGB values ​​and thus generate balls of multiple colors. These modifications are included in the file provided in the next section.

When an object changes its position gradually, it creates the illusion of being in motion. To achieve this effect, we will use registers to save the position of the edge coordinates and update these registers with each new screen scan (60 times per second). It is therefore necessary to generate a clock signal at 60Hz (**refr_tick**).


* Add the **pong_graph_animate.vhd** and **pong_top_an.vhd** files to the project. Set the latter as the top module (**Set as Top**).

* Open the **pong_graph_animate.vhd** file and observe the code that allows you to generate the round ball. In lines 59 to 79 the ROM memory is defined. In lines 163 to 165 it is defined that the ball must be **ON** (that is, the color of the ball must be sent to the screen) only when:

    * the sweep is inside the square that defines the ball (**sq_ball_on='1'**)
    * the bit stored in memory is at a high logic level (**rom_bit='1'**)
    
    
* Note the code that allows the **refr_tick** signal to be generated on lines 103 to 108.

The bar will now be controlled by push buttons, so it can be moved up and down. The constants that define the edge coordinates must then be changed to signs (**bar_y_t** for the top and **bar_y_b** for the bottom) and a register (**bar_y_reg**) must be created that stores the current coordinate of the top edge. Whenever the external buttons are pressed (btn(1) and btn(0) to move up and down, respectively), the value in this register must be increased or decreased, at a frequency of 60Hz.

* Note the difference between the code that defines the static wall (lines 109 to 118) and the code that defines the dynamic bar (lines 119 to 144).

* Note that the increment value is defined by the constant **BAR_V** (lines 137 and 139). The magnitude of this increase will define the speed we want to impart to the bar. The bar should stop when it reaches the top or bottom of the screen.


The ball animation is a little more complex. In this case, two registers are needed, one for the horizontal position and the other for the vertical position (**ball_x_reg** and **ball_y_reg**). On the other hand, it is necessary to decompose the speed into a component on the **X** axis and another on the **Y** axis, to allow the ball to change direction when hitting the bar or the limits of the screen. These components can take on positive or negative values, defined by the constants **BALL_V_P** and **BALL_V_N**, respectively. 


* Observe the ball animation code (lines 144 to 194). The movement is printed on the rectangular ball, in the same way as it is done with the bar, with the difference that we now have movement in both axes using the **ball_x_reg** and **ball_y_reg** registers, updated in lines 169 to 174.

* The speed of the ball in each axis (**x_delta_reg** and **y_delta_reg**) will depend on the direction we want to give the ball with each bounce. Look at the code on lines 176 to 194.

* Implement this circuit in Basys 3. Remember that you need to update the input/output pin file.

* Program the board and use the push buttons to move the bar.



______________

## Step 3 - Tile-mapped layout for text. ##

### Step 3.1 ### 

To be able to represent text on a VGA monitor, it is first necessary to build a memory with the font characters, as is done in the **font_rom.vhd** file .

* Add this file to the project. Open it and look at the code. Note that it is a ROM memory (defined as a constant), which allows storing 8-bit data (DATA_WIDTH: integer:=8) and has 2048 memory locations (ADDR_WIDTH: integer:=11). Each **ASCII** character occupies 16 memory locations


To check the operation of the ROM, we will use a circuit that allows you to view all the characters on the screen. The code for this circuit is provided in **font_test_gen.vhd**. The **font_test_top.vhd** file combines these circuits with the screen timing circuit and clock generator, allowing you to implement the test circuit on the development board.

* Add these files to the project. Set the **font_test_top.vhd** circuit as the top module (*Set as Top*) and modify the input/output pin file. Test the circuit and observe the characters on the screen.




_________

## Challenge ##

### Challenge 1 - Text Mode ###

Change the code in the **font_test_gen.vhd** file to increase the font size. See how in section 13.2.5 (page 297) of the reference book:

<div>
<img src="lab7/fig9.png" width="500"/>
</div>

### Challenge 2 - Animate Mode ###

Change the code in the **pong_graph_animate.vhd** file to increase the size of the ball by 4 times.

### Challenge 3 - Pong Game ###

Create a new project named PONG and add all the fonts provided on the UC page. Note that you will also have to add a Clocking Wizard module here to obtain a 50MHz signal, as in step 1.1 of this lab work. Program the board and have fun!
